In [1]:
import pandas as pd
from utils.config import FOLDER_DATA
from file_management import LIVE_PROCESSED_TABLE

# The most up to date scrapped csv version is used
test = pd.read_csv(f'{FOLDER_DATA}/{LIVE_PROCESSED_TABLE}') if LIVE_PROCESSED_TABLE else None

scrapped_films_202305171705.csv
processed_films_202305171705.csv


In [10]:
test.isnull().sum()

rank            0
title           0
duration        0
genres          0
director        0
cast            0
synopsis        0
rating          0
duration_min    0
dtype: int64